## Estimating lifespan normative models

This notebook provides a complete walkthrough for an analysis of normative modelling using your own dataset. The tutorial is based on the teaching material at: https://github.com/CharFraza/CPC_ML_tutorial/. Here you can also find more normative modelling tutorials.
Training and testing data is provided for this tutorial. However, the idea is that you could subsitute our provided training and testing datasets for you own dataset (as long as it matches the same format!)

First, if necessary, we install PCNtoolkit (note: this tutorial requires at least version 0.20)

In [ ]:
# Make sure to click the restart runtime button at the
# bottom of this code blocks' output (after you run the cell)
! pip install pcntoolkit==0.28

Then we import the required libraries

In [2]:
! git clone https://github.com/CharFraza/CPC_ML_tutorial.git

Cloning into 'CPC_ML_tutorial'...


In [1]:
# we need to be in the CPC_ML_tutorial folder when we import the libraries in the code block below,
# because there is a function called nm_utils that is in this folder that we need to import
import os
os.chdir('CPC_ML_tutorial')

In [2]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns

from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d

Now, we configure the locations in which the data are stored.

**Notes:**
- The data are assumed to be in CSV format and will be loaded as pandas dataframes
- Generally the raw data will be in a different location to the analysis
- The data can have arbitrary columns but some are required by the script, i.e. 'age', 'sex' and 'site', plus the phenotypes you wish to estimate (see below)

In [ ]:
# where the raw data are stored
data_dir = 'data'

# where the analysis takes place
root_dir = 'D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

Now we load the data.

We will load one pandas dataframe for the training set and one dataframe for the test set. We also configure a list of site ids.

In [4]:
"""work1 code"""
train = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\GMV_health_ins2.csv')
test = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\GMV_disease_ins2.csv')
Site = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\Site_chu.csv')
Site2 = Site[['eid', 'site']]
label = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\GMV_labels.csv')
train.columns = train.columns[:1].tolist() + label['Description'].to_list() + train.columns[-2:].tolist()
test.columns = test.columns[:1].tolist() + label['Description'].to_list() + test.columns[-2:].tolist()
train = pd.merge(train, Site2, on='eid')
test = pd.merge(test, Site2, on='eid')

In [6]:
train.to_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\train_data.csv')
test.to_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\test_data.csv')

In [22]:
import pandas as pd
df_tr = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1_gmv\train_data.csv', index_col=0)
df_te = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1_gmv\test_data.csv', index_col=0)

# extract a list of unique site ids from the training set
site_ids = sorted(set(df_tr['site'].to_list()))

### Configure which models to fit

Next, we load the image derived phenotypes (IDPs) which we will process in this analysis. This is effectively just a list of columns in your dataframe. Here we estimate normative models for the left hemisphere, right hemisphere and cortical structures.

In [23]:
# we choose here to process all idps. Uncomment lines 2-7 (and comment line 11) to run models for the whole brain, but we suggest just starting with several ROIs
#os.chdir(root_dir)
#!wget -nc https://raw.githubusercontent.com/CharFraza/CPC_ML_tutorial/master/data/task1_phenotypes.txt
#with open(os.path.join(root_dir,'task1_phenotypes.txt')) as f:
#  idp_ids = f.read().splitlines()
#for idx, ele in enumerate(idp_ids):
#        idp_ids[idx] = ele.replace('\t', '')

# we could also just specify a list of IDPs. Use this line to run just 2 models (not the whole brain)...this is a good place to start. If you have time,
# you can uncomment the above line and run the whole brain models. Be sure to comment out this line if you uncomment the above line.
idp_ids = df_tr.columns[4:]

### Configure model parameters

Now, we configure some parameters for the regression model we use to fit the normative model. Here we will use a 'warped' Bayesian linear regression model. To model non-Gaussianity, we select a sin arcsinh warp and to model non-linearity, we stick with the default value for the basis expansion (a cubic b-spline basis set with 5 knot points). Since we are sticking with the default value, we do not need to specify any parameters for this, but we do need to specify the limits. We choose to pad the input by a few years either side of the input range. We will also set a couple of options that control the estimation of the model

For further details about the likelihood warping approach, see [Fraza et al 2021](https://www.biorxiv.org/content/10.1101/2021.04.05.438429v1).

In [24]:
# check the min & max age of the dataset, use this info to update the xmin & xmax variables in the code block below.
df_tr['age'].describe()

count    24014.000000
mean        63.297021
std          7.446508
min         45.166667
25%         57.333333
50%         63.583333
75%         69.083333
max         82.250000
Name: age, dtype: float64

In [25]:
# which data columns do we wish to use as covariates?
# You could add additional covariates from your own dataset here that you wish to use as predictors.
# However, for this tutorial today we will keep it simple and just use age & sex.
# Maybe discuss with your partner ideas you have for other covariates you would like to include.
cols_cov = ['age','sex']

# which warping function to use? We can set this to None in order to fit a vanilla Gaussian noise model
warp =  'WarpSinArcsinh'
#warp = None

# limits for cubic B-spline basis
# check the min & max ages of the dataframes, add 5 to the max
# and subtract 5 from the min and adjust these variables accordingly
xmin = 45 # set this variable
xmax = 83 # set this variable

# Do we want to force the model to be refit every time?
# When training normative model from scratch like we are doing in this notebook (not re-using a pre-trained model),
# this variable should be = True
force_refit = True

# Absolute Z treshold above which a sample is considered to be an outlier (without fitting any model)
outlier_thresh = 7

### Fit the models

Now we fit the models. This involves looping over the IDPs we have selected. We will use a module from PCNtoolkit to set up the design matrices, containing the covariates, fixed effects for site and nonlinear basis expansion.

In [26]:
for idp_num, idp in enumerate(idp_ids):
    print('Running IDP', idp_num, idp, ':')

    idp_dir = os.path.join(out_dir, idp)
    os.makedirs(idp_dir, exist_ok=True)
    os.chdir(idp_dir)

    # extract the response variables for training and test set
    y_tr = df_tr[idp].to_numpy()
    y_te = df_te[idp].to_numpy()

    # remove gross outliers and implausible values
    yz_tr = (y_tr - np.mean(y_tr)) / np.std(y_tr)
    yz_te = (y_te - np.mean(y_te)) / np.std(y_te)
    nz_tr = np.bitwise_and(np.abs(yz_tr) < outlier_thresh, y_tr > 0)
    nz_te = np.bitwise_and(np.abs(yz_te) < outlier_thresh, y_te > 0)
    y_tr = y_tr[nz_tr]
    y_te = y_te[nz_te]

    eid = df_te['eid'][nz_te]
    eid_file_te = os.path.join(idp_dir, 'eid.txt')
    np.savetxt(eid_file_te, eid, fmt='%d')

    # write out the response variables for training and test
    resp_file_tr = os.path.join(idp_dir, 'resp_tr.txt')
    resp_file_te = os.path.join(idp_dir, 'resp_te.txt')
    np.savetxt(resp_file_tr, y_tr)
    np.savetxt(resp_file_te, y_te)

    # configure the design matrix
    X_tr = create_design_matrix(df_tr[cols_cov].loc[nz_tr],
                                site_ids = df_tr['site'].loc[nz_tr],
                                basis = 'bspline',
                                p = 3,
                                nknots = 4,
                                xmin = xmin,
                                xmax = xmax)
    X_te = create_design_matrix(df_te[cols_cov].loc[nz_te],
                                site_ids = df_te['site'].loc[nz_te],
                                basis = 'bspline',
                                p = 3,
                                nknots = 4,
                                xmin = xmin,
                                xmax = xmax)

    # configure and save the covariates
    cov_file_tr = os.path.join(idp_dir, 'cov_bspline_tr.txt')
    cov_file_te = os.path.join(idp_dir, 'cov_bspline_te.txt')
    np.savetxt(cov_file_tr, X_tr)
    np.savetxt(cov_file_te, X_te)

    if not force_refit and os.path.exists(os.path.join(idp_dir, 'Models', 'NM_0_0_estimate.pkl')):
        print('Making predictions using a pre-existing model...')
        suffix = 'predict'

        # Make prdictsion with test data
        predict(cov_file_te,
                alg='blr',
                respfile=resp_file_te,
                model_path=os.path.join(idp_dir,'Models'),
                outputsuffix=suffix)
    else:
        print('Estimating the normative model...')
        estimate(cov_file_tr, resp_file_tr, testresp=resp_file_te,
                 testcov=cov_file_te, alg='blr', optimizer = 'powell',
                 savemodel=True, warp=warp, warp_reparam=True)  #'cg','powell','nelder-mead','l-bfgs-b'
        suffix = 'estimate'

Running IDP 0 Frontal Pole (left) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Frontal Pole (left)\resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Optimization terminated successfully.
         Current function value: 221234.046625
         Iterations: 3
         Function evaluations: 174
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 1 Frontal Pole (right) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Frontal Pole (right)\resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Optimization terminated successfully.
         Current function value: 224139.002076
         Iterations: 3
         Function evaluations: 181
Saving model meta-data...
Eva

c:\Users\1\.conda\envs\pcn\lib\site-packages\pcntoolkit\model\bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.05466e-20): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)


Optimization terminated successfully.
         Current function value: 212427.400475
         Iterations: 2
         Function evaluations: 126
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 5 Superior Frontal Gyrus (right) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Superior Frontal Gyrus (right)\resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Optimization terminated successfully.
         Current function value: 210407.990839
         Iterations: 2
         Function evaluations: 125
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 6 Middle Frontal Gyrus (left) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Middle Frontal Gyrus (left)\resp_tr.txt
Estim

c:\Users\1\.conda\envs\pcn\lib\site-packages\pcntoolkit\model\bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.83593e-21): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)


Optimization terminated successfully.
         Current function value: 212516.388354
         Iterations: 2
         Function evaluations: 124
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 7 Middle Frontal Gyrus (right) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Middle Frontal Gyrus (right)\resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters


...

Optimization terminated successfully.
         Current function value: 164606.741863
         Iterations: 2
         Function evaluations: 120
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 117 VI Cerebellum (right) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\VI Cerebellum (right)\resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Optimization terminated successfully.
         Current function value: 198562.170476
         Iterations: 2
         Function evaluations: 125
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 118 Crus I Cerebellum (left) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Crus I Cerebellum (left)\resp_tr.txt
Estimating model  1 of 1


c:\Users\1\.conda\envs\pcn\lib\site-packages\pcntoolkit\model\bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.5568e-20): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)


Optimization terminated successfully.
         Current function value: 211323.388538
         Iterations: 2
         Function evaluations: 129
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 121 Crus II Cerebellum (left) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Crus II Cerebellum (left)\resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Optimization terminated successfully.
         Current function value: 202733.225648
         Iterations: 2
         Function evaluations: 124
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 122 Crus II Cerebellum (vermis) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\Crus II Cerebellum (vermis)\resp_tr.txt
Estimating 

c:\Users\1\.conda\envs\pcn\lib\site-packages\pcntoolkit\model\bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.01528e-20): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
c:\Users\1\.conda\envs\pcn\lib\site-packages\pcntoolkit\model\bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.9652e-19): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)


Optimization terminated successfully.
         Current function value: 188053.635296
         Iterations: 2
         Function evaluations: 129
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 125 VIIb Cerebellum (vermis) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\VIIb Cerebellum (vermis)\resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Optimization terminated successfully.
         Current function value: 113205.298057
         Iterations: 2
         Function evaluations: 115
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 126 VIIb Cerebellum (right) :
Estimating the normative model...
Processing data in D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test\models\test\VIIb Cerebellum (right)\resp_tr.txt
Estimating model  1 o

## Questions
1. Can you explain the covariance matrix and it's elements?
2. How do you change the order of the spline function?
3. How do you change the number of knots?


In [27]:
print(pd.DataFrame(X_te))

        0          1    2    3    4    5         6         7         8   \
0      1.0  55.250000  0.0  0.0  0.0  1.0  0.006945  0.408239  0.496501   
1      1.0  69.166667  1.0  1.0  0.0  0.0  0.000000  0.000195  0.216505   
2      1.0  66.916667  0.0  0.0  0.0  1.0  0.000000  0.004906  0.326466   
3      1.0  76.666666  1.0  1.0  0.0  0.0  0.000000  0.000000  0.020833   
4      1.0  73.500000  1.0  1.0  0.0  0.0  0.000000  0.000000  0.070313   
...    ...        ...  ...  ...  ...  ...       ...       ...       ...   
11037  1.0  68.166667  1.0  1.0  0.0  0.0  0.000000  0.001251  0.263903   
11038  1.0  71.416666  0.0  1.0  0.0  0.0  0.000000  0.000000  0.127457   
11039  1.0  57.083334  1.0  1.0  0.0  0.0  0.000098  0.285959  0.569259   
11040  1.0  70.750000  0.0  0.0  1.0  0.0  0.000000  0.000000  0.150754   
11041  1.0  67.666667  1.0  0.0  0.0  1.0  0.000000  0.002333  0.288648   

             9         10        11  
0      0.088315  0.000000  0.000000  
1      0.596211  0.1870

### Compute error metrics

In this section we compute the following error metrics for all IDPs (all evaluated on the test set): assess the goodness of fit between the predicted probabilities of a model and the actual observed outcomes.
- Negative log likelihood (NLL): NLL assesses the goodness of fit between the predicted probabilities of a model and the actual observed outcomes. In this case, it measures the discrepancy between the predicted probabilities of the model for the IDPs (Independent Data Points) and the actual outcomes on the test set.
- Explained variance (EV): EV assesses how much of the total variation in the dependent variable (IDP) is explained by the independent variables. In the context of this analysis, it quantifies the extent to which the independent variables account for the variability observed in the IDPs on the test set.
- Mean standardized log loss (MSLL): MSLL takes into account both the mean error and the estimated prediction variance. It is used to evaluate the performance of the model, and in this case, a lower MSLL indicates a better-fitting model for the IDPs on the test set.
- Bayesian information criteria (BIC): BIC is a model selection criterion that balances the goodness of fit to the data with the model's complexity. It penalizes models with higher flexibility and aims to find the best trade-off. Lower BIC scores indicate models that better explain the IDPs on the test set while considering the model complexity.
- Skew and Kurtosis of the Z-distribution: Skewness and kurtosis are statistical measures used to assess the shape and characteristics of a distribution. They provide information about how well the warping function performed in terms of capturing the departure from a normal distribution for the IDPs.

For more information on the different model selection criteria see [Fraza et al 2021](https://www.biorxiv.org/content/10.1101/2021.04.05.438429v1)

In [28]:
# initialise dataframe we will use to store quantitative metrics
blr_metrics = pd.DataFrame(columns = ['eid', 'NLL', 'EV', 'MSLL', 'BIC','Skew','Kurtosis'])

for idp_num, idp in enumerate(idp_ids):
    print('Running IDP', idp_num, idp, ':')

    idp_dir = os.path.join(out_dir, idp)

    # load the predictions and true data. We use a custom function that ensures 2d arrays
    # equivalent to: y = np.loadtxt(filename); y = y[:, np.newaxis]
    yhat_te = load_2d(os.path.join(idp_dir, 'yhat_' + suffix + '.txt'))
    s2_te = load_2d(os.path.join(idp_dir, 'ys2_' + suffix + '.txt'))
    y_te = load_2d(os.path.join(idp_dir, 'resp_te.txt'))

    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle)

    # compute error metrics
    if warp is None:
        metrics = evaluate(y_te, yhat_te)

        # compute MSLL manually as a sanity check
        y_tr_mean = np.array( [[np.mean(y_tr)]] )
        y_tr_var = np.array( [[np.var(y_tr)]] )
        MSLL = compute_MSLL(y_te, yhat_te, s2_te, y_tr_mean, y_tr_var)
    else:
        warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1]
        W = nm.blr.warp

        # warp predictions
        med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
        med_te = med_te[:, np.newaxis]

        # evaluation metrics
        metrics = evaluate(y_te, med_te)

        # compute MSLL manually
        y_te_w = W.f(y_te, warp_param)
        y_tr_w = W.f(y_tr, warp_param)
        y_tr_mean = np.array( [[np.mean(y_tr_w)]] )
        y_tr_var = np.array( [[np.var(y_tr_w)]] )
        MSLL = compute_MSLL(y_te_w, yhat_te, s2_te, y_tr_mean, y_tr_var)

    Z = np.loadtxt(os.path.join(idp_dir, 'Z_' + suffix + '.txt'))
    [skew, sdskew, kurtosis, sdkurtosis, semean, sesd] = calibration_descriptives(Z)

    BIC = len(nm.blr.hyp) * np.log(y_tr.shape[0]) + 2 * nm.neg_log_lik

    blr_metrics.loc[len(blr_metrics)] = [idp, nm.neg_log_lik, metrics['EXPV'][0],
                                         MSLL[0], BIC, skew, kurtosis]

display(blr_metrics)

blr_metrics.to_csv(os.path.join(out_dir,'blr_metrics.csv'))

Running IDP 0 Frontal Pole (left) :
Running IDP 1 Frontal Pole (right) :
Running IDP 2 Insular Cortex (left) :
Running IDP 3 Insular Cortex (right) :
Running IDP 4 Superior Frontal Gyrus (left) :
Running IDP 5 Superior Frontal Gyrus (right) :
Running IDP 6 Middle Frontal Gyrus (left) :
Running IDP 7 Middle Frontal Gyrus (right) :
Running IDP 8 Inferior Frontal Gyrus, pars triangularis (left) :
Running IDP 9 Inferior Frontal Gyrus, pars triangularis (right) :
Running IDP 10 Inferior Frontal Gyrus, pars opercularis (left) :
Running IDP 11 Inferior Frontal Gyrus, pars opercularis (right) :
Running IDP 12 Precentral Gyrus (left) :
Running IDP 13 Precentral Gyrus (right) :
Running IDP 14 Temporal Pole (left) :
Running IDP 15 Temporal Pole (right) :
Running IDP 16 Superior Temporal Gyrus, anterior division (left) :
Running IDP 17 Superior Temporal Gyrus, anterior division (right) :
Running IDP 18 Superior Temporal Gyrus, posterior division (left) :
Running IDP 19 Superior Temporal Gyrus, pos

,eid,NLL,EV,MSLL,BIC,Skew,Kurtosis
0,Frontal Pole (left),221234.046625,0.223566,-39829.032814,442508.438486,0.322949,0.899796
1,Frontal Pole (right),224139.002076,0.213622,-51456.624548,448318.349388,0.290688,0.704564
2,Insular Cortex (left),188227.856624,0.173576,-2629.565851,376496.058484,0.363168,0.928431
3,Insular Cortex (right),187936.206394,0.176169,-2617.937611,375912.758024,0.363670,1.089612
4,Superior Frontal Gyrus (left),212427.400475,0.086296,-8710.737801,424895.146186,0.265439,0.320915
...,...,...,...,...,...,...,...
134,IX Cerebellum (vermis),141358.423576,0.026455,-0.081836,282757.192388,0.401589,0.510240
135,IX Cerebellum (right),178742.114149,0.038048,-217.490783,357524.573534,0.446999,0.260030
136,X Cerebellum (left),137790.919540,0.130080,-0.141607,275622.184316,0.093361,0.923937
137,X Cerebellum (vermis),123703.223578,0.066344,-4.261242,247446.792392,0.533758,0.663412


## Questions to discuss
1. Model selection: Which model selection criteria would you use to choose the optimal model?
2. Model flexibility: What happens when you change the warping or B-spline settings?
3. Bias-variance tradeoff: How would you consider the bias-variance tradeoff when deciding the models parameters?
4. Which independent variables do you think are important to add to the normative model?
5. Are there other model selection criteria that you think should be considered?

## Suggested further readings

1. [PCNtoolkit Background](https://pcntoolkit.readthedocs.io/en/latest/pages/pcntoolkit_background.html)
2. [Conceptualizing mental disorders as deviations from normative functioning](https://www.nature.com/articles/s41380-019-0441-1)
3. [Understanding Heterogeneity in Clinical Cohorts Using Normative Models: Beyond Case-Control Studies
](https://www.sciencedirect.com/science/article/pii/S0006322316000020)


In [32]:
# which sex do we want to plot?
sex = 1 # 1 = male 0 = female
if sex == 1:
    clr = 'blue';
else:
    clr = 'red'

# create dummy data for visualisation
print('configuring dummy data ...')
xx = np.arange(xmin, xmax, 0.5)
X0_dummy = np.zeros((len(xx), 2))
X0_dummy[:,0] = xx
X0_dummy[:,1] = sex

# create the design matrix
X_dummy = create_design_matrix(X0_dummy, xmin=xmin, xmax=xmax, site_ids=None ,p=3 ,nknots=4 ,all_sites=site_ids)

# save the dummy covariates
cov_file_dummy = os.path.join(out_dir,'cov_bspline_dummy_mean.txt')
np.savetxt(cov_file_dummy, X_dummy)

configuring dummy data ...


In [33]:
sns.set(style='white')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

for idp_num, idp in enumerate(idp_ids):
    #idp_short = extract_relevant_part(idp)
    print('Running IDP', idp_num, idp, ':')

    idp_dir = os.path.join(out_dir, idp)
    os.chdir(idp_dir)

    # load the true data points
    yhat_te = load_2d(os.path.join(idp_dir, 'yhat_estimate.txt'))
    s2_te = load_2d(os.path.join(idp_dir, 'ys2_estimate.txt'))
    y_te = load_2d(os.path.join(idp_dir, 'resp_te.txt'))

    plt.figure(figsize=(3, 3))

    # set up the covariates for the dummy data
    print('Making predictions with dummy covariates (for visualisation)')
    yhat, s2 = predict(cov_file_dummy,
                       alg = 'blr',
                       respfile = None,
                       model_path = os.path.join(idp_dir,'Models'),
                       outputsuffix = '_dummy')

    # load the normative model
    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle)

    # get the warp and warp parameters
    W = nm.blr.warp
    warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1]

    # first, we warp predictions for the true data and compute evaluation metrics
    med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
    med_te = med_te[:, np.newaxis]
    print('metrics:', evaluate(y_te, med_te))

    # then, we warp dummy predictions to create the plots
    med, pr_int = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param)

    # extract the different variance components to visualise
    beta, junk1, junk2 = nm.blr._parse_hyps(nm.blr.hyp, X_dummy)
    s2n = 1/beta # variation (aleatoric uncertainty)
    s2s = s2-s2n # modelling uncertainty (epistemic uncertainty)

    # plot the data points
    y_te_rescaled_all = np.zeros_like(y_te)
    for sid, site in enumerate(site_ids):
        # plot the true test data points
        if all(elem in site_ids for elem in site_ids):
            # all data in the test set are present in the training set

            # first, we select the data points belonging to this particular site
            idx = np.where(np.bitwise_and(X_te[:,2] == sex, X_te[:, sid + len(cols_cov) + 1] != 0))[0]
            if len(idx) == 0:
                print('No data for site', sid, site, 'skipping...')
                continue
            idx = idx[idx < len(y_te)]
            # then directly adjust the data
            idx_dummy = np.bitwise_and(X_dummy[:,1] > X_te[idx,1].min(), X_dummy[:,1] < X_te[idx,1].max())
            y_te_rescaled = y_te[idx] - np.median(y_te[idx]) + np.median(med[idx_dummy])
        else:
            # we need to adjust the data based on the adaptation dataset

            # first, select the data point belonging to this particular site
            idx = np.where(np.bitwise_and(X_te[:,2] == sex, (df_te['site'] == site).to_numpy()))[0]

            # load the adaptation data
            y_ad = load_2d(os.path.join(idp_dir, 'resp_ad.txt'))
            X_ad = load_2d(os.path.join(idp_dir, 'cov_bspline_ad.txt'))
            idx_a = np.where(np.bitwise_and(X_ad[:,2] == sex, (df_tr['site'] == site).to_numpy()))[0]
            if len(idx) < 2 or len(idx_a) < 2:
                print('Insufficent data for site', sid, site, 'skipping...')
                continue

            # adjust and rescale the data
            y_te_rescaled, s2_rescaled = nm.blr.predict_and_adjust(nm.blr.hyp,
                                                                   X_ad[idx_a,:],
                                                                   np.squeeze(y_ad[idx_a]),
                                                                   Xs=None,
                                                                   ys=np.squeeze(y_te[idx]))
        # plot the (adjusted) data points
        plt.scatter(X_te[idx,1], y_te_rescaled, s=5, color=clr, alpha = 0.1)

    # plot the median of the dummy data
    plt.plot(xx, med, clr)

    # fill the gaps in between the centiles
    junk, pr_int25 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.05,0.95])
    #junk, pr_int99 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25[:,0], pr_int25[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int95[:,0], pr_int95[:,1], alpha = 0.1,color=clr)
    #plt.fill_between(xx, pr_int99[:,0], pr_int99[:,1], alpha = 0.1,color=clr)

    # make the width of each centile proportional to the epistemic uncertainty
    junk, pr_int25l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.05,0.95])
    #junk, pr_int99l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.01,0.99])
    junk, pr_int25u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.05,0.95])
    #junk, pr_int99u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25l[:,0], pr_int25u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,0], pr_int95u[:,0], alpha = 0.3,color=clr)
    #plt.fill_between(xx, pr_int99l[:,0], pr_int99u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int25l[:,1], pr_int25u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,1], pr_int95u[:,1], alpha = 0.3,color=clr)
    #plt.fill_between(xx, pr_int99l[:,1], pr_int99u[:,1], alpha = 0.3,color=clr)

    # plot actual centile lines
    plt.plot(xx, pr_int25[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int25[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,1],color=clr, linewidth=0.5)
    #plt.plot(xx, pr_int99[:,0],color=clr, linewidth=0.5)
    #plt.plot(xx, pr_int99[:,1],color=clr, linewidth=0.5)

    plt.xlabel('Age', fontweight='bold')
    plt.ylabel('GMV', fontweight='bold')
    plt.title(idp, fontweight='bold')
    plt.xlim((45,83))
    plt.savefig(os.path.join(idp_dir, 'centiles_' + str(sex) + '.svg'), bbox_inches='tight', transparent=True, format='svg', dpi=300)
    plt.close()
    #plt.show()

os.chdir(out_dir)

Running IDP 0 Frontal Pole (left) :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
metrics: {'RMSE': array([2403.77375021]), 'Rho': array([0.4728507]), 'pRho': array([0.]), 'SMSE': array([0.78173834]), 'EXPV': array([0.22356602])}
Running IDP 1 Frontal Pole (right) :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
metrics: {'RMSE': array([2695.36365967]), 'Rho': array([0.46242062]), 'pRho': array([0.]), 'SMSE': array([0.79127982]), 'EXPV': array([0.2136217])}
Running IDP 2 Insular Cortex (left) :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
metrics: {'RMSE': array([624.88955276]), 'Rho': array([0.41663582]), 'pRho': array([0.]), 'SMSE': array([0.83128461]), 'EXPV': array([0.17357577])}
Running IDP 3 Insular Cortex (right) :
Making predictions with dumm

In [ ]:
import os
import pandas as pd

base_path = 'D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test/models/test'

final_df = pd.DataFrame()

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        z_file_path = os.path.join(folder_path, 'Z_estimate.txt')
        eid_file_path = os.path.join(folder_path, 'eid.txt')
        
        if os.path.isfile(z_file_path) and os.path.isfile(eid_file_path):
            z_values = pd.read_csv(z_file_path, header=None)
            eid = pd.read_csv(eid_file_path, header=None)
            
            df = pd.concat([eid, z_values], axis=1)
            df.columns = ['eid', folder]
            
            if final_df.empty:
                final_df = df
            else:
                final_df = pd.merge(final_df, df, on='eid', how='outer')

final_df.to_csv('D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test/combined_data.csv', index=False)

In [14]:
data = pd.read_csv('D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/fa_test/combined_data.csv')

data['Count_Greater_Than_2.6'] = data.iloc[:,1:].apply(lambda row: (row > 2.6).sum(), axis=1)
data['Count_Less_Than_-2.6'] = data.iloc[:,1:].apply(lambda row: (row < -2.6).sum(), axis=1)

print(max(data['Count_Less_Than_-2.6']))

32


In [15]:
data.head()

,eid,anterior corona radiata (left),anterior corona radiata (right),anterior limb of internal capsule (left),anterior limb of internal capsule (right),body of corpus callosum,cerebral peduncle (left),cerebral peduncle (right),cingulum cingulate gyrus (left),cingulum cingulate gyrus (right),...,superior fronto-occipital fasciculus (left),superior fronto-occipital fasciculus (right),superior longitudinal fasciculus (left),superior longitudinal fasciculus (right),tapetum (left),tapetum (right),uncinate fasciculus (left),uncinate fasciculus (right),Count_Greater_Than_2.6,Count_Less_Than_-2.6
0,1000314,0.811903,1.287647,0.785331,0.816113,0.769631,1.306642,2.015253,0.126916,0.312918,...,0.355215,0.257909,1.210605,0.939175,-1.370862,-0.684752,1.238076,2.068952,0,0
1,1000388,-0.571057,-0.456034,-0.622337,-0.548966,-0.704128,0.224381,0.382133,0.825040,0.132577,...,0.109107,-0.374743,-1.138934,-0.880273,0.617824,0.380678,-0.118191,-0.888942,0,0
2,1000571,-1.095148,-0.971031,-1.707692,-1.401137,-0.404326,-0.372279,-0.522517,-0.951638,-0.496043,...,-1.303315,-1.119341,-1.476685,-1.711894,-0.755207,0.148188,-1.082473,0.272134,0,0
3,1001060,1.922104,1.323251,0.152737,-0.427638,1.147957,0.205889,0.526655,-0.145259,1.231099,...,0.674429,0.031246,0.025373,0.939138,-0.555793,-0.153721,1.313984,1.418973,0,0
4,1001238,-0.800706,-1.360128,-0.990315,-0.868130,-0.517029,-0.775753,-0.603963,0.013948,0.710524,...,-0.258492,-0.107724,-0.426724,-0.022979,-1.635695,-1.160070,-0.014045,0.222692,0,0


In [16]:
data.to_csv('D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/fa_test/ex_data.csv')

In [14]:
data = pd.read_csv('D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/fa_test/combined_data.csv')
#greater_than_2_6 = (data.iloc[:,1:] > 2.6).sum()
#less_than_minus_2_6 = (data.iloc[:,1:] < -2.6).sum()
mean_z_score = data.iloc[:, 1:].mean(axis=0)

In [22]:
data = pd.read_csv('D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test/combined_data.csv')
#greater_than_2_6 = (data.iloc[:,1:] > 2.6).sum()
#less_than_minus_2_6 = (data.iloc[:,1:] < -2.6).sum()
mean_z_score = data.iloc[:, 1:].mean(axis=0)

summary_df = pd.DataFrame({
    'Brain_Region': data.iloc[:,1:].columns,
    'Mean_z_score': mean_z_score,
    #'Count_Greater_Than_2.6': greater_than_2_6,
    #'Count_Less_Than_-2.6': less_than_minus_2_6
})

In [23]:
summary_df.to_csv('D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test/z_GMV.csv', index=False)

In [20]:
summary_df.sort_values(by='Mean_z_score')

,Brain_Region,Mean_z_score
posterior thalamic radiation (left),posterior thalamic radiation (left),-0.169246
superior cerebellar peduncle (left),superior cerebellar peduncle (left),-0.167460
superior cerebellar peduncle (right),superior cerebellar peduncle (right),-0.167156
posterior thalamic radiation (right),posterior thalamic radiation (right),-0.155457
anterior limb of internal capsule (right),anterior limb of internal capsule (right),-0.154877
anterior limb of internal capsule (left),anterior limb of internal capsule (left),-0.150736
anterior corona radiata (left),anterior corona radiata (left),-0.147218
fornix cres+stria terminalis (left),fornix cres+stria terminalis (left),-0.145814
superior fronto-occipital fasciculus (right),superior fronto-occipital fasciculus (right),-0.144114
anterior corona radiata (right),anterior corona radiata (right),-0.143267
